In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from integral_timber_joints.planning import load_pddlstream

Using pddlstream from C:\Users\harry\Documents\code_ws\pb_ws\coop_assembly\external\pddlstream\pddlstream\__init__.py
Using strips (pyplanners) from C:\Users\harry\Documents\code_ws\pb_ws\coop_assembly\external\pyplanners\strips\__init__.py


## Parse ITJ process

In [3]:
import os
import time
from termcolor import cprint
import pybullet_planning as pp
from pybullet_planning import elapsed_time
from integral_timber_joints.planning.parsing import parse_process, save_process_and_movements, \
    get_process_path, save_process

In [4]:
# parse options
design_dir = '211010_CantiBox' # '210916_SymbolicPlanning' # '210419_AnticlasticShelter' # '210605_ScrewdriverTestProcess' # 210419_AnticlasticShelter
problem = 'CantiBoxLeft_process.json' # 'nine_pieces_process.json' #'shelter_process.json' # 'shelter_process.json' pavilion_process.json' # 'twelve_pieces_process.json'
problem_subdir = '.'

In [5]:
process = parse_process(design_dir, problem, subdir=problem_subdir)

# Double check entire solution is valid
for beam_id in process.assembly.sequence:
    if not process.dependency.beam_all_valid(beam_id):
        print('Yay')
        process.dependency.compute_all(beam_id)
        assert process.dependency.beam_all_valid(beam_id)

Process json parsed from c:\users\harry\dropbox (mit)\code_ws_dropbox\itj_ws\integral_timber_joints\external\itj_design_study\211010_CantiBox\CantiBoxLeft_process.json


## Solve

In [7]:
discrete_planner = {
    'search': 'eager', # eager | lazy
    # lazy might be faster but the solution quality might be low
    'evaluator': 'greedy',
    'heuristic': 'goal', # goal | add | ff
    #'heuristic': ['ff', get_bias_fn(element_from_index)],
    'successors': 'all', # all | random
    #'successors': get_order_fn(element_from_index),
    # ? ordering scaffolds can come here
    # TODO: confirm that this is working correctly
}

In [25]:
from integral_timber_joints.planning.robot_setup import load_RFL_world
from integral_timber_joints.planning.run import set_initial_state

# * Connect to path planning backend and initialize robot parameters
# viewer or diagnosis or view_states or watch or step_sim,
client, robot, _ = load_RFL_world(viewer=1, verbose=False)
set_initial_state(client, robot, process, disable_env=False, reinit_tool=False)

In [31]:
client.disconnect()

In [30]:
from pddlstream.algorithms.downward import set_cost_scale, parse_action
from pddlstream.algorithms.meta import solve
from pddlstream.utils import INF
from pddlstream.language.constants import print_plan, is_plan

from integral_timber_joints.planning.state import set_state
from integral_timber_joints.planning.pddlstream_definitions.problem import get_pddlstream_problem
from integral_timber_joints.planning.pddlstream_definitions.postprocessing import print_itj_pddl_plan

debug = 0
options = {
    'avoid_collisions' : False,
    'verbose' : 0,
    'debug' : debug,
    'diagnosis' : 0,
    'distance_threshold' : 0.02,
}

set_state(client, robot, process, process.initial_state, initialize=False)
pddlstream_problem = get_pddlstream_problem(client, process, robot, use_partial_order=1, 
                                            debug=0, reset_to_home=1, 
                                            consider_transition=0, options=options)

# print('Init:', pddlstream_problem.init)
print()
print('Goal:', pddlstream_problem.goal)
print()

costs = True
set_cost_scale(1)

start_time = time.time()
with pp.LockRenderer(not debug):
    solution = solve(pddlstream_problem, algorithm='incremental', #adaptive
                     max_time=60,
                     unit_costs=True,
                     max_planner_time=300, 
                     debug=0, verbose=0,)
                     # planner=discrete_planner)
cprint('Total solve time: {:.3f}'.format(elapsed_time(start_time)), 'blue')

plan, cost, evaluations = solution
plan_success = is_plan(plan)
cprint('Planning {}'.format('succeeds' if plan_success else 'fails'), 'green' if plan_success else 'red')

print('-'*10)
print_itj_pddl_plan(plan, 0)


Goal: ('and', ('Assembled', 'b0'), ('Assembled', 'b1'), ('Assembled', 'b2'), ('Assembled', 'b3'), ('Assembled', 'b5'), ('Assembled', 'b6'), ('Assembled', 'b7'), ('Assembled', 'b8'), ('Assembled', 'b4'), ('AtRack', 'c1'), ('AtRack', 'c2'), ('AtRack', 'c3'), ('AtRack', 'c4'), ('AtRack', 'g1'), ('AtRack', 'g2'), ('AtRack', 'g3'))

Iteration: 1 | Complexity: 0 | Calls: 0 | Evaluations: 278 | Solved: False | Cost: inf | Search Time: 0.001 | Sample Time: 0.000 | Time: 0.001
Initial: State{Atom atpose(v37, v38), Atom atpose(v40, v41), Atom atpose(v43, v44), Atom atpose(v46, v47), Atom atpose(v59, v60), Atom atpose(v62, v63), Atom atpose(v65, v66), Atom notoolatjoint(v1, v13), Atom notoolatjoint(v1, v21), Atom notoolatjoint(v1, v9), Atom notoolatjoint(v13, v1), Atom notoolatjoint(v13, v5), Atom notoolatjoint(v17, v21), Atom notoolatjoint(v17, v33), Atom notoolatjoint(v21, v1), Atom notoolatjoint(v21, v17), Atom notoolatjoint(v25, v29), Atom notoolatjoint(v25, v33), Atom notoolatjoint(v29, v25

## Debug

# Sequence

In [23]:
place_actions = [action for action in plan if action.name == 'place_element_on_structure']
element_sequence = [ac.args[0] for ac in place_actions]

In [24]:
process.assembly.sequence

['b0', 'b1', 'b2', 'b3', 'b5', 'b6', 'b7', 'b8', 'b4']

In [25]:
element_sequence

['b0', 'b1', 'b2', 'b3', 'b5', 'b6', 'b7', 'b8', 'b4']

In [26]:
place_actions

[Action(name='place_element_on_structure', args=('b0', Frame(Point(19016.343, 9160.006, 279.218), Vector(0.000, 0.000, 1.000), Vector(-1.000, 0.000, 0.000)), Frame(Point(25496.239, 7136.104, 579.218), Vector(0.000, 1.000, 0.000), Vector(-1.000, 0.000, 0.000)), 'g1', Frame(Point(23439.739, 4480.771, 990.818), Vector(-0.000, 1.000, 0.000), Vector(1.000, 0.000, -0.000)), @conf335)),
 Action(name='place_element_on_structure', args=('b1', Frame(Point(19016.343, 8260.006, 3361.742), Vector(0.000, 0.000, -1.000), Vector(-1.000, 0.000, -0.000)), Frame(Point(25496.239, 7068.375, 579.218), Vector(0.000, 1.000, 0.000), Vector(-1.000, 0.000, 0.000)), 'g1', Frame(Point(23439.739, 4480.771, 990.818), Vector(-0.000, 1.000, 0.000), Vector(1.000, 0.000, -0.000)), @conf338)),
 Action(name='place_element_on_structure', args=('b2', Frame(Point(19016.343, 9455.953, 1810.165), Vector(-0.000, -0.986, -0.165), Vector(-1.000, 0.000, 0.000)), Frame(Point(25496.239, 7251.367, 579.218), Vector(0.000, 1.000, 0.000

# Visualize sequence in a pybullet window

In [10]:
from integral_timber_joints.planning.pddlstream_definitions.postprocessing import actions_from_pddlstream_plan

actions_from_pddlstream_plan(process, plan, verbose=1)

operator_load_element_on_rack b0 (frm)
pick_tool_from_rack g2 (frm) (tf) Conf-
pick_element_from_rack b0 (frm) (tf) g2 (tf) (command)
operator_load_element_on_rack b1 (frm)
place_element_on_structure b0 (frm) (tf) g2 (tf) Conf-
Beam id: b0
|- Operator load Beam ('b0') for pickup
|- Pick PG1000 ('g2') from Storage
|- Pick Beam ('b0') from PickupPoint using Gripper(g2)
|- Operator load Beam ('b1') for pickup
|- Place Beam ('b0') to final location without clamps
pick_element_from_rack b1 (frm) (tf) g2 (tf) (command)
operator_load_element_on_rack b2 (frm)
place_element_on_structure b1 (frm) (tf) g2 (tf) Conf-
Beam id: b1
|- Pick Beam ('b1') from PickupPoint using Gripper(g2)
|- Operator load Beam ('b2') for pickup
|- Place Beam ('b1') to final location without clamps
pick_element_from_rack b2 (frm) (tf) g2 (tf) (command)
operator_load_element_on_rack b3 (frm)
place_element_on_structure b2 (frm) (tf) g2 (tf) Conf-
Beam id: b2
|- Pick Beam ('b2') from PickupPoint using Gripper(g2)
|- Operato

1

In [105]:
from integral_timber_joints.planning.robot_setup import load_RFL_world
from integral_timber_joints.planning.run import set_initial_state

# * Connect to path planning backend and initialize robot parameters
# viewer or diagnosis or view_states or watch or step_sim,
client, robot, _ = load_RFL_world(viewer=1, verbose=False)
set_initial_state(client, robot, process, disable_env=False, reinit_tool=False)

In [18]:
client.disconnect()

In [11]:
from integral_timber_joints.planning.state import set_state
from integral_timber_joints.process import RoboticMovement, RobotClampAssemblyProcess

for m in process.movements:
    print(m.short_summary)
    start_state = process.get_movement_start_scene(m)
    end_state = process.get_movement_end_scene(m)  
    set_state(client, robot, process, end_state, initialize=False)
#     pp.wait_if_gui('End state.')
    pp.wait_for_duration(0.1)
print('Finished.')

OperatorLoadBeamMovement(#A0_M0, Opeartor Load Beam (b0) to Pickup Location)
RoboticFreeMovement(#A0_M0, Free Move reach Storage Approach Frame of PG1000 ('g2'), to get tool., traj 0)
RoboticLinearMovement(#A0_M1, Linear Advance to Storage Frame of PG1000 ('g2'), to get tool., traj 0)
RoboticDigitalOutput(#A0_M2, Toolchanger Lock PG1000 ('g2'))
RoboticDigitalOutput(#A0_M3, PG1000 ('g2') Open Gripper to release itself from storage pad.)
RoboticLinearMovement(#A0_M4, Linear Retract after getting PG1000 ('g2') from storage., traj 0)
RoboticFreeMovement(#A0_M0, Free Move to reach Pickup Approach Frame of Beam ('b0'), traj 0)
RoboticDigitalOutput(#A0_M1, Gripper ('g2') Open Gripper before gripping Beam ('b0'))
RoboticLinearMovement(#A0_M2, Linear Advance to Storage Frame of Beam ('b0'), traj 1)
RoboticDigitalOutput(#A0_M3, Gripper ('g2') Close Gripper to grip Beam ('b0'))
RoboticLinearMovement(#A0_M4, Linear Retract after picking up Beam ('b0'), traj 1)
OperatorLoadBeamMovement(#A0_M0, Opea

## Construct symbolic PDDL problem

In [6]:
from integral_timber_joints.planning.pddlstream_definitions.problem import get_pddlstream_problem
from integral_timber_joints.planning.pddlstream_definitions import ITJ_PDDLSTREAM_DEF_DIR

Use Track IK: False


In [11]:
import json
process_sym_data = process.to_symbolic_problem_data()
debug_problem_name = problem.split(".")[0] + "_symbolic.json"
debug_save_path = r'C:\Users\harry\Documents\code_ws\pb_ws\debug_pyplanners\itj'
with open(os.path.join(debug_save_path, debug_problem_name), 'w') as f:
    json.dump(process_sym_data, f, indent=True, sort_keys=True)

In [10]:
from integral_timber_joints.planning.pddlstream_definitions.caelan.run import get_itj_pddl_problem_from_json

debug_pddl_problem = get_itj_pddl_problem_from_json(debug_problem_name, use_partial_order=True, debug=True)

print()
print('Goal:', debug_pddl_problem.goal)
print()

costs = True
set_cost_scale(1)

solution = solve(debug_pddl_problem, algorithm='adaptive', #incremental
                 max_time=60,
                 unit_costs=True,
                 max_planner_time=300, 
                 debug=0, verbose=0) #, planner=discrete_planner)

plan, cost, evaluations = solution
plan_success = is_plan(plan)
cprint('Planning {}'.format('succeeds' if plan_success else 'fails'), 'green' if plan_success else 'red')

print('-'*10)
print_plan(plan)

ModuleNotFoundError: No module named 'integral_timber_joints.planning.pddlstream_definitions.caelan'

# Misc

In [25]:
# from pddlstream.algorithms.algorithm import parse_problem
# from pddlstream.algorithms.constraints import PlanConstraints
# from pddlstream.algorithms.downward import get_problem, task_from_domain_problem
# from pddlstream.algorithms.meta import examine_instantiated

# pddlstream_problem = get_pddlstream_problem(process, use_partial_order=True, debug=False,
#                                             reset_to_home=False)

# print(pddlstream_problem.goal)

# results, instantiated = examine_instantiated(pddlstream_problem, unit_costs=1, verbose=0, debug=1)